# 🏦 Bank Statement Processor - PRODUCTION v2.0

## 📋 Configuration
| Resource | ID |
|----------|----|
| **Spreadsheet ID** | `1XuvPyWNhB3WAOMHDO9wXkZ5AO36Cce13PH8PAojG9Eo` |
| **Input Folder ID** | `1vVQC5F8ZrKZnqv5QAWcye5VKGxCQ1c3q` |
| **Processed Folder ID** | `198Xe7BBn3ibRgXoUnOOz2NMWRcPgNtQj` |

---
**Run cells 1-4 in order. Cell 4 processes ALL PDFs.**

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# CELL 1: INSTALL DEPENDENCIES
# ═══════════════════════════════════════════════════════════════════════════════

!pip install -q gspread google-auth google-auth-oauthlib google-auth-httplib2
!pip install -q pandas numpy matplotlib seaborn
!pip install -q pdfplumber PyPDF2
!pip install -q google-api-python-client

print('✅ All packages installed successfully!')

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# CELL 2: AUTHENTICATION & COMPLETE CONFIGURATION
# ═══════════════════════════════════════════════════════════════════════════════

# Standard library imports
import io
import os
import re
import uuid
from datetime import datetime
from typing import Dict, List, Optional, Any, Tuple

# Third-party imports
import pandas as pd
import numpy as np
import pdfplumber
import gspread
from google.colab import auth
from google.auth import default
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload

# ═══════════════════════════════════════════════════════════════════════════════
# AUTHENTICATE WITH GOOGLE
# ═══════════════════════════════════════════════════════════════════════════════
print('🔐 Authenticating with Google...')
auth.authenticate_user()

# Get credentials and build services
creds, project = default()
gc = gspread.authorize(creds)
drive_service = build('drive', 'v3', credentials=creds)

# ═══════════════════════════════════════════════════════════════════════════════
# 🔧 YOUR PROJECT CONFIGURATION - EDIT THESE IF NEEDED
# ═══════════════════════════════════════════════════════════════════════════════
CONFIG = {
    'SPREADSHEET_ID': '1XuvPyWNhB3WAOMHDO9wXkZ5AO36Cce13PH8PAojG9Eo',
    'INPUT_FOLDER_ID': '1vVQC5F8ZrKZnqv5QAWcye5VKGxCQ1c3q',
    'PROCESSED_FOLDER_ID': '198Xe7BBn3ibRgXoUnOOz2NMWRcPgNtQj',
    'OUTPUT_FOLDER_ID': '1GQxr8YiFnm23YM78k77DYzT0tF1CIQMp',
    'ARCHIVE_FOLDER_ID': '1KQ31p9k1QSgh2dgteEWuDhrs0x3la-4n',
    'TRANSACTIONS_SHEET': 'Transactions',
    'LOG_SHEET': 'Processing Log'
}

# Connect to spreadsheet
print('📊 Connecting to spreadsheet...')
try:
    spreadsheet = gc.open_by_key(CONFIG['SPREADSHEET_ID'])
    print(f'✅ Connected to: {spreadsheet.title}')
    print(f'📋 Sheets: {[ws.title for ws in spreadsheet.worksheets()]}')
except Exception as e:
    print(f'❌ Connection failed: {e}')
    raise SystemExit('Cannot continue without spreadsheet connection')

print('\n✅ Authentication and configuration complete!')

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# CELL 3: COMPLETE HELPER FUNCTIONS (ALL FULLY IMPLEMENTED)
# ═══════════════════════════════════════════════════════════════════════════════

def list_pdfs_in_folder(folder_id: str) -> List[Dict[str, Any]]:
    """
    List all PDF files in a Google Drive folder.
    
    Args:
        folder_id: Google Drive folder ID
        
    Returns:
        List of file dictionaries with id, name, size, modifiedTime
    """
    query = f"'{folder_id}' in parents and mimeType='application/pdf' and trashed=false"
    try:
        results = drive_service.files().list(
            q=query,
            fields='files(id, name, size, modifiedTime)',
            orderBy='name',
            pageSize=100
        ).execute()
        return results.get('files', [])
    except Exception as e:
        print(f'❌ Error listing PDFs: {e}')
        return []


def download_pdf(file_id: str, filename: str) -> str:
    """
    Download a PDF from Google Drive to local storage.
    
    Args:
        file_id: Google Drive file ID
        filename: Name for the local file
        
    Returns:
        Local file path
    """
    request = drive_service.files().get_media(fileId=file_id)
    fh = io.BytesIO()
    downloader = MediaIoBaseDownload(fh, request)
    
    done = False
    while not done:
        status, done = downloader.next_chunk()
    
    fh.seek(0)
    local_path = f'/content/{filename}'
    
    with open(local_path, 'wb') as f:
        f.write(fh.read())
    
    return local_path


def extract_text_from_pdf(pdf_path: str) -> str:
    """
    Extract all text from a PDF file using pdfplumber.
    
    Args:
        pdf_path: Path to the PDF file
        
    Returns:
        Extracted text as a single string
    """
    text = ''
    try:
        with pdfplumber.open(pdf_path) as pdf:
            for page_num, page in enumerate(pdf.pages):
                page_text = page.extract_text()
                if page_text:
                    text += page_text + '\n'
    except Exception as e:
        print(f'   ⚠️ PDF extraction error: {e}')
    
    return text


def detect_bank_type(text: str) -> str:
    """
    Detect the bank type from statement text.
    
    Args:
        text: Extracted PDF text
        
    Returns:
        Bank identifier string
    """
    lower_text = text.lower()
    
    if 'capital one' in lower_text or '360 checking' in lower_text or 'capitalone' in lower_text:
        return 'CAPITAL_ONE'
    elif 'chase' in lower_text or 'jpmorgan' in lower_text:
        return 'CHASE'
    elif 'bank of america' in lower_text or 'bofa' in lower_text:
        return 'BANK_OF_AMERICA'
    elif 'wells fargo' in lower_text:
        return 'WELLS_FARGO'
    
    return 'UNKNOWN'


def parse_capital_one_statement(text: str, source_file: str) -> List[Dict[str, Any]]:
    """
    Parse Capital One bank statement with multiple regex patterns.
    This is a robust parser that tries multiple patterns to extract transactions.
    
    Args:
        text: Extracted statement text
        source_file: Name of the source PDF file
        
    Returns:
        List of transaction dictionaries
    """
    transactions = []
    lines = text.split('\n')
    
    # Month abbreviation to number mapping
    month_map = {
        'jan': 1, 'feb': 2, 'mar': 3, 'apr': 4, 'may': 5, 'jun': 6,
        'jul': 7, 'aug': 8, 'sep': 9, 'oct': 10, 'nov': 11, 'dec': 12,
        'january': 1, 'february': 2, 'march': 3, 'april': 4,
        'june': 6, 'july': 7, 'august': 8, 'september': 9,
        'october': 10, 'november': 11, 'december': 12
    }
    
    # Multiple patterns for Capital One statements
    patterns = [
        # Pattern 1: "Sep 3 Description Credit|Debit +$123.45 $1,234.56"
        r'^(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[.\s]+?(\d{1,2})\s+(.+?)\s+(Credit|Debit)?\s*([+-]?\$?[\d,]+\.\d{2})\s+\$?([\d,]+\.\d{2})\s*$',
        
        # Pattern 2: "Sep 3 Description $123.45 $1,234.56" (without Credit/Debit)
        r'^(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[.\s]+?(\d{1,2})\s+(.+?)\s+([+-]?\$?[\d,]+\.\d{2})\s+\$?([\d,]+\.\d{2})\s*$',
        
        # Pattern 3: "09/03 Description $123.45" (MM/DD format)
        r'^(\d{1,2})/(\d{1,2})\s+(.+?)\s+([+-]?\$?[\d,]+\.\d{2})\s*$',
        
        # Pattern 4: "Sep 03 Description 123.45" (no $ sign)
        r'^(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[.\s]+?(\d{1,2})\s+(.+?)\s+([+-]?[\d,]+\.\d{2})\s*$',
        
        # Pattern 5: More flexible - date at start, amount at end
        r'^(\d{1,2})[/\-](\d{1,2})[/\-]?(\d{2,4})?\s+(.+?)\s+([+-]?\$?[\d,]+\.\d{2})\s*$',
    ]
    
    # Words to skip (headers, footers, summaries)
    skip_keywords = [
        'opening balance', 'closing balance', 'beginning balance', 'ending balance',
        'page', 'statement', 'customer service', 'capitalone.com', 'capital one',
        'account summary', 'account number', 'routing number', 'total deposits',
        'total withdrawals', 'interest paid', 'fees charged', 'member fdic',
        'transaction history', 'account activity', 'daily balance', 'date description',
        'amount balance', 'deposits/credits', 'withdrawals/debits'
    ]
    
    current_year = datetime.now().year
    
    for line in lines:
        line = line.strip()
        
        # Skip empty lines
        if not line or len(line) < 10:
            continue
        
        # Skip header/footer lines
        lower_line = line.lower()
        if any(keyword in lower_line for keyword in skip_keywords):
            continue
        
        # Try each pattern
        for pattern_idx, pattern in enumerate(patterns):
            match = re.match(pattern, line, re.IGNORECASE)
            if match:
                groups = match.groups()
                
                try:
                    # Parse based on which pattern matched
                    if pattern_idx in [0, 1, 3]:  # Month name patterns
                        month_str = groups[0].lower()[:3]
                        month = month_map.get(month_str, 1)
                        day = int(groups[1])
                        description = groups[2].strip()
                        
                        # Get amount (handle different group positions)
                        if pattern_idx == 0:  # Has Credit/Debit and balance
                            amount_str = groups[4]
                            balance_str = groups[5]
                            trans_type = groups[3] if groups[3] else ''
                        elif pattern_idx == 1:  # Has balance but no Credit/Debit
                            amount_str = groups[3]
                            balance_str = groups[4]
                            trans_type = ''
                        else:  # Pattern 3 - just amount
                            amount_str = groups[3]
                            balance_str = ''
                            trans_type = ''
                            
                    elif pattern_idx == 2:  # MM/DD pattern
                        month = int(groups[0])
                        day = int(groups[1])
                        description = groups[2].strip()
                        amount_str = groups[3]
                        balance_str = ''
                        trans_type = ''
                        
                    else:  # Pattern 4 - flexible date
                        month = int(groups[0])
                        day = int(groups[1])
                        description = groups[3].strip() if len(groups) > 3 else groups[2].strip()
                        amount_str = groups[-1]
                        balance_str = ''
                        trans_type = ''
                    
                    # Clean and parse amount
                    amount_clean = re.sub(r'[\$,\s]', '', amount_str)
                    amount = float(amount_clean)
                    
                    # Determine if debit (negative)
                    if '-' in amount_str or trans_type.lower() == 'debit':
                        amount = -abs(amount)
                    elif '+' in amount_str or trans_type.lower() == 'credit':
                        amount = abs(amount)
                    
                    # Parse balance if present
                    balance = ''
                    if balance_str:
                        balance_clean = re.sub(r'[\$,\s]', '', balance_str)
                        try:
                            balance = float(balance_clean)
                        except ValueError:
                            balance = ''
                    
                    # Create date string
                    try:
                        trans_date = datetime(current_year, month, day)
                        date_str = trans_date.strftime('%Y-%m-%d')
                    except ValueError:
                        date_str = f'{current_year}-{month:02d}-{day:02d}'
                    
                    # Skip if description is too short or looks like a header
                    if len(description) < 3:
                        continue
                    
                    # Create transaction record
                    transaction = {
                        'id': str(uuid.uuid4())[:8],
                        'date': date_str,
                        'description': description[:150],
                        'category': '',
                        'amount': amount,
                        'type': 'Income' if amount >= 0 else 'Expense',
                        'balance': balance,
                        'bank': 'Capital One',
                        'account': '',
                        'period': '',
                        'processed': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                        'source_file': source_file
                    }
                    
                    transactions.append(transaction)
                    break  # Found a match, move to next line
                    
                except (ValueError, IndexError) as e:
                    # Pattern matched but parsing failed, try next pattern
                    continue
    
    return transactions


def parse_generic_statement(text: str, source_file: str) -> List[Dict[str, Any]]:
    """
    Generic fallback parser for unknown bank formats.
    Looks for lines with dates and amounts.
    
    Args:
        text: Extracted statement text
        source_file: Name of the source PDF file
        
    Returns:
        List of transaction dictionaries
    """
    transactions = []
    lines = text.split('\n')
    
    # Generic pattern: any date-like thing at start, description, amount at end
    pattern = r'^([\d/\-]+)\s+(.+?)\s+([+-]?\$?[\d,]+\.\d{2})\s*$'
    
    for line in lines:
        line = line.strip()
        if not line or len(line) < 10:
            continue
            
        match = re.search(pattern, line)
        if match:
            date_str, description, amount_str = match.groups()
            
            try:
                amount = float(re.sub(r'[\$,\s]', '', amount_str))
                if '-' in amount_str:
                    amount = -abs(amount)
                
                transactions.append({
                    'id': str(uuid.uuid4())[:8],
                    'date': date_str,
                    'description': description.strip()[:150],
                    'category': '',
                    'amount': amount,
                    'type': 'Income' if amount >= 0 else 'Expense',
                    'balance': '',
                    'bank': 'Unknown',
                    'account': '',
                    'period': '',
                    'processed': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                    'source_file': source_file
                })
            except ValueError:
                continue
    
    return transactions


def write_transactions_to_sheet(
    spreadsheet: gspread.Spreadsheet, 
    transactions: List[Dict[str, Any]], 
    sheet_name: str = 'Transactions'
) -> int:
    """
    Write transactions to Google Sheet.
    
    Args:
        spreadsheet: gspread Spreadsheet object
        transactions: List of transaction dictionaries
        sheet_name: Name of the target sheet
        
    Returns:
        Number of rows written
    """
    if not transactions:
        return 0
    
    try:
        # Get or create sheet
        try:
            sheet = spreadsheet.worksheet(sheet_name)
        except gspread.WorksheetNotFound:
            sheet = spreadsheet.add_worksheet(title=sheet_name, rows=5000, cols=12)
            headers = ['ID', 'Date', 'Description', 'Category', 'Amount', 'Type', 
                       'Balance', 'Bank', 'Account', 'Period', 'Processed', 'Source']
            sheet.update(values=[headers], range_name='A1:L1')
        
        # Convert transactions to rows
        rows = []
        for t in transactions:
            rows.append([
                str(t.get('id', '')),
                str(t.get('date', '')),
                str(t.get('description', '')),
                str(t.get('category', '')),
                t.get('amount', 0),
                str(t.get('type', '')),
                t.get('balance', ''),
                str(t.get('bank', '')),
                str(t.get('account', '')),
                str(t.get('period', '')),
                str(t.get('processed', '')),
                str(t.get('source_file', ''))
            ])
        
        # Get next empty row
        all_values = sheet.get_all_values()
        next_row = len(all_values) + 1
        
        # Write data (using correct argument order: values first, range second)
        end_row = next_row + len(rows) - 1
        range_name = f'A{next_row}:L{end_row}'
        sheet.update(values=rows, range_name=range_name)
        
        return len(rows)
        
    except Exception as e:
        print(f'❌ Error writing to sheet: {e}')
        return 0


def move_file_to_folder(file_id: str, dest_folder_id: str) -> bool:
    """
    Move a file to a different folder in Google Drive.
    
    Args:
        file_id: Google Drive file ID
        dest_folder_id: Destination folder ID
        
    Returns:
        True if successful, False otherwise
    """
    try:
        # Get current parents
        file = drive_service.files().get(
            fileId=file_id,
            fields='parents'
        ).execute()
        
        previous_parents = ','.join(file.get('parents', []))
        
        # Move to new folder
        drive_service.files().update(
            fileId=file_id,
            addParents=dest_folder_id,
            removeParents=previous_parents,
            fields='id, parents'
        ).execute()
        
        return True
        
    except Exception as e:
        print(f'   ⚠️ Error moving file: {e}')
        return False


print('✅ All helper functions loaded!')

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# CELL 4: PROCESS ALL PDFs - MAIN EXECUTION
# ═══════════════════════════════════════════════════════════════════════════════

def process_all_pdfs() -> Dict[str, Any]:
    """
    Main function to process all PDFs in the input folder.
    Downloads PDFs, extracts text, parses transactions, writes to sheet,
    and moves processed files.
    
    Returns:
        Summary dictionary with processing results
    """
    print('🚀 STARTING BATCH PDF PROCESSING')
    print('=' * 60)
    
    start_time = datetime.now()
    
    # Initialize counters
    results = {
        'total_files': 0,
        'processed_files': 0,
        'total_transactions': 0,
        'errors': [],
        'files_processed': []
    }
    
    # List all PDFs in input folder
    print('\n📁 Scanning input folder for PDFs...')
    pdf_files = list_pdfs_in_folder(CONFIG['INPUT_FOLDER_ID'])
    results['total_files'] = len(pdf_files)
    
    if not pdf_files:
        print('⚠️ No PDF files found in input folder!')
        print(f'   Folder ID: {CONFIG["INPUT_FOLDER_ID"]}')
        return results
    
    print(f'✅ Found {len(pdf_files)} PDF file(s)\n')
    
    # Process each PDF
    for idx, pdf in enumerate(pdf_files):
        file_name = pdf['name']
        file_id = pdf['id']
        
        print(f'\n📄 [{idx + 1}/{len(pdf_files)}] {file_name}')
        print('-' * 50)
        
        try:
            # Step 1: Download PDF
            print('   📥 Downloading...')
            local_path = download_pdf(file_id, file_name)
            print(f'   ✅ Downloaded to {local_path}')
            
            # Step 2: Extract text
            print('   📝 Extracting text...')
            extracted_text = extract_text_from_pdf(local_path)
            print(f'   ✅ Extracted {len(extracted_text)} characters')
            
            # Step 3: Detect bank type
            bank_type = detect_bank_type(extracted_text)
            print(f'   🏦 Bank detected: {bank_type}')
            
            # Step 4: Parse transactions
            print('   🔍 Parsing transactions...')
            if bank_type == 'CAPITAL_ONE':
                transactions = parse_capital_one_statement(extracted_text, file_name)
            else:
                transactions = parse_generic_statement(extracted_text, file_name)
            
            print(f'   ✅ Found {len(transactions)} transactions')
            
            # Step 5: Write to sheet
            if transactions:
                print('   💾 Writing to sheet...')
                count = write_transactions_to_sheet(spreadsheet, transactions)
                results['total_transactions'] += count
                print(f'   ✅ Written {count} rows to sheet')
            else:
                print('   ⚠️ No transactions to write')
                # Show sample of extracted text for debugging
                print('   📝 Sample text (first 500 chars):')
                print(extracted_text[:500])
            
            # Step 6: Move to processed folder
            print('   📦 Moving to processed folder...')
            if move_file_to_folder(file_id, CONFIG['PROCESSED_FOLDER_ID']):
                print('   ✅ Moved successfully')
            
            # Step 7: Cleanup local file
            if os.path.exists(local_path):
                os.remove(local_path)
            
            results['processed_files'] += 1
            results['files_processed'].append({
                'name': file_name,
                'transactions': len(transactions),
                'bank': bank_type
            })
            
        except Exception as e:
            error_msg = f'{file_name}: {str(e)}'
            results['errors'].append(error_msg)
            print(f'   ❌ ERROR: {e}')
            continue
    
    # Calculate duration
    duration = (datetime.now() - start_time).total_seconds()
    
    # Print summary
    print('\n' + '=' * 60)
    print('📊 PROCESSING COMPLETE')
    print('=' * 60)
    print(f'   📁 Files found:      {results["total_files"]}')
    print(f'   ✅ Files processed:  {results["processed_files"]}')
    print(f'   📝 Total transactions: {results["total_transactions"]}')
    print(f'   ❌ Errors:           {len(results["errors"])}')
    print(f'   ⏱️ Duration:         {duration:.1f} seconds')
    
    if results['errors']:
        print('\n⚠️ Errors encountered:')
        for error in results['errors']:
            print(f'   - {error}')
    
    print(f'\n🔗 View spreadsheet: https://docs.google.com/spreadsheets/d/{CONFIG["SPREADSHEET_ID"]}')
    
    return results


# ═══════════════════════════════════════════════════════════════════════════════
# EXECUTE MAIN PROCESSING
# ═══════════════════════════════════════════════════════════════════════════════
processing_results = process_all_pdfs()

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# CELL 5: DEBUG - VIEW EXTRACTED TEXT (Run if 0 transactions found)
# ═══════════════════════════════════════════════════════════════════════════════

def debug_pdf_extraction(pdf_index: int = 0):
    """
    Debug function to see what text is being extracted from a PDF.
    Use this if you're getting 0 transactions to see the actual format.
    
    Args:
        pdf_index: Index of PDF to debug (0 = first file)
    """
    print('🔍 DEBUG: PDF Text Extraction')
    print('=' * 60)
    
    # List PDFs
    pdf_files = list_pdfs_in_folder(CONFIG['INPUT_FOLDER_ID'])
    
    if not pdf_files:
        print('❌ No PDFs found in input folder')
        return
    
    if pdf_index >= len(pdf_files):
        print(f'❌ Invalid index. Only {len(pdf_files)} files available.')
        return
    
    pdf = pdf_files[pdf_index]
    print(f'\n📄 File: {pdf["name"]}')
    print('-' * 50)
    
    # Download and extract
    local_path = download_pdf(pdf['id'], pdf['name'])
    text = extract_text_from_pdf(local_path)
    
    print(f'\n📊 Total characters extracted: {len(text)}')
    print(f'📊 Total lines: {len(text.split(chr(10)))}')
    
    print('\n' + '=' * 60)
    print('📝 FIRST 100 LINES OF TEXT:')
    print('=' * 60)
    
    lines = text.split('\n')
    for i, line in enumerate(lines[:100]):
        if line.strip():
            print(f'{i:4d}: {line}')
    
    # Cleanup
    if os.path.exists(local_path):
        os.remove(local_path)
    
    print('\n' + '=' * 60)
    print('💡 Look for lines that contain transactions')
    print('   (date + description + amount pattern)')


# Uncomment to run debug:
# debug_pdf_extraction(0)  # Debug first PDF

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# CELL 6: LOAD & ANALYZE TRANSACTIONS FROM SHEET
# ═══════════════════════════════════════════════════════════════════════════════

try:
    # Load transactions sheet
    transactions_sheet = spreadsheet.worksheet(CONFIG['TRANSACTIONS_SHEET'])
    data = transactions_sheet.get_all_values()
    
    if len(data) > 1:
        # Create DataFrame
        df = pd.DataFrame(data[1:], columns=data[0])
        print(f'✅ Loaded {len(df)} transactions\n')
        
        # Convert Amount to numeric
        df['Amount'] = pd.to_numeric(df['Amount'], errors='coerce')
        
        # Financial summary
        total_income = df[df['Amount'] > 0]['Amount'].sum()
        total_expenses = df[df['Amount'] < 0]['Amount'].sum()
        net_flow = total_income + total_expenses
        
        print('📊 FINANCIAL SUMMARY')
        print('=' * 40)
        print(f'💰 Total Income:    ${total_income:,.2f}')
        print(f'💸 Total Expenses:  ${abs(total_expenses):,.2f}')
        print(f'📈 Net Cash Flow:   ${net_flow:,.2f}')
        print('=' * 40)
        
        # Show recent transactions
        print('\n📋 Recent Transactions:')
        display(df.tail(15))
        
        # By bank breakdown
        if 'Bank' in df.columns:
            print('\n🏦 By Bank:')
            bank_summary = df.groupby('Bank')['Amount'].agg(['count', 'sum'])
            bank_summary.columns = ['Count', 'Total']
            display(bank_summary)
    else:
        print('⚠️ No transaction data found in sheet')
        
except Exception as e:
    print(f'❌ Error loading transactions: {e}')